In [ ]:
#############################################################################
# zlib License
#
# (C) 2024 Cristóvão Beirão da Cruz e Silva <cbeiraod@cern.ch>
#
# This software is provided 'as-is', without any express or implied
# warranty.  In no event will the authors be held liable for any damages
# arising from the use of this software.
#
# Permission is granted to anyone to use this software for any purpose,
# including commercial applications, and to alter it and redistribute it
# freely, subject to the following restrictions:
#
# 1. The origin of this software must not be misrepresented; you must not
#    claim that you wrote the original software. If you use this software
#    in a product, an acknowledgment in the product documentation would be
#    appreciated but is not required.
# 2. Altered source versions must be plainly marked as such, and must not be
#    misrepresented as being the original software.
# 3. This notice may not be removed or altered from any source distribution.
#############################################################################

In [ ]:
from pathlib import Path
import sqlite3
import pandas
import datetime
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import mplhep
from math import floor

from SEUhelper import *

import sys, os
path2add = os.path.normpath(os.path.abspath(os.path.join(os.path.curdir, os.path.pardir, 'TestBeam')))
if (not (path2add in sys.path)) :
    sys.path.append(path2add)

In [ ]:
# This notebook is for analysing the data from the 28th of June 2024, the SEU testing performed at UCLouvain w/ KULeuven

# One board was used for this testing: ET2_Bare_7

start_time = datetime.datetime(2024, 6, 28, 1, 6, 55)
end_time = datetime.datetime(2024, 6, 28, 8, 0, 0)

base_directory = Path("/Volumes/AnalysisData/SEU/SEU_28Jun2024/")

power_file = base_directory/"PowerHistory_v2.sqlite"
config_directory = base_directory/"ChipConfig"
output_dir = base_directory/"AnalysisOutput"
output_dir.mkdir(exist_ok=True)

chip_names = [
    "ET2p01_Bare_7",
]

power_connections = {
    "ET2.01 Bare Board 7": {
        "Analog": "Analog",
        "Digital": "Digital",
    },
}

run_info = [
    {
        "name": "Run_1",
        #"extra_begin": datetime.datetime(2024, 1, 27, 9, 10, 0),  # Optional: Used if we want to add an additional time for the per run plotting
        "start": datetime.datetime(2024, 6, 28, 1, 44, 24),
        "stop": datetime.datetime(2024, 6, 28, 1, 56, 25),
        "boards": ["ET2.01 Bare Board 7"],
        "board_channels": [0],
        "pre_config_times": [datetime.datetime(2024, 6, 28, 1, 44, 0)],
        "intermediate_config_times": [
            [
                datetime.datetime(2024, 6, 28, 1, 46, 13),
                datetime.datetime(2024, 6, 28, 1, 47, 42),
                datetime.datetime(2024, 6, 28, 1, 50, 4),
                datetime.datetime(2024, 6, 28, 1, 54, 25)
            ],
                                      ],
        "post_config_times": [datetime.datetime(2024, 6, 28, 1, 56, 34)],
        "config_before": True,
        #"extra_run_info": ["Run_SEUJan24_16"],  # Optional: Used if we want to add some info from another run in the extra run list
        "fluence": 1E+7,
        "fluence_type": "Kr",
        "pixel_TMR": "off", #"mixed",  # Technically it is mixed, but will use off
    },
    {
        "name": "Run_2",
        "start": datetime.datetime(2024, 6, 28, 2, 4, 46),
        "stop": datetime.datetime(2024, 6, 28, 2, 16, 48),
        "boards": ["ET2.01 Bare Board 7"],
        "board_channels": [0],
        "pre_config_times": [datetime.datetime(2024, 6, 28, 2, 4, 43)],
        "intermediate_config_times": [
            [
                datetime.datetime(2024, 6, 28, 2, 6, 39),
                datetime.datetime(2024, 6, 28, 2, 8, 37),
                datetime.datetime(2024, 6, 28, 2, 10, 38),
                datetime.datetime(2024, 6, 28, 2, 13, 0),
            ],
                                      ],
        "post_config_times": [datetime.datetime(2024, 6, 28, 2, 16, 57)],
        "config_before": True,
        "fluence": 1E+7,
        "fluence_type": "Kr",
        "pixel_TMR": "on",
    },
    {
        "name": "Run_3",
        "start": datetime.datetime(2024, 6, 28, 2, 21, 31),
        "stop": datetime.datetime(2024, 6, 28, 2, 33, 35),
        "boards": ["ET2.01 Bare Board 7"],
        "board_channels": [0],
        "pre_config_times": [datetime.datetime(2024, 6, 28, 2, 20, 41)],
        "intermediate_config_times": [
            [
                datetime.datetime(2024, 6, 28, 2, 24, 51),
                datetime.datetime(2024, 6, 28, 2, 29, 14),
            ],
                                      ],
        "post_config_times": [datetime.datetime(2024, 6, 28, 2, 34, 12)],
        "config_before": False,
        "fluence": 1E+7,
        "fluence_type": "Kr",
        "pixel_TMR": "on",
    },
    {
        "name": "Run_4",
        "start": datetime.datetime(2024, 6, 28, 2, 36, 15),
        "stop": datetime.datetime(2024, 6, 28, 2, 48, 21), # Wrong in log
        "boards": ["ET2.01 Bare Board 7"],
        "board_channels": [0],
        "pre_config_times": [datetime.datetime(2024, 6, 28, 2, 36, 1)],
        "intermediate_config_times": [
            [
                datetime.datetime(2024, 6, 28, 2, 40, 3),
                datetime.datetime(2024, 6, 28, 2, 42, 24),
                datetime.datetime(2024, 6, 28, 2, 45, 37),
            ],
                                      ],
        "post_config_times": [datetime.datetime(2024, 6, 28, 2, 48, 30)],
        "config_before": False,
        "fluence": 1E+7,
        "fluence_type": "Kr",
        "pixel_TMR": "on",
    },
    {
        "name": "Run_5",
        "start": datetime.datetime(2024, 6, 28, 3, 4, 2),
        "stop": datetime.datetime(2024, 6, 28, 3, 22, 11),
        "boards": ["ET2.01 Bare Board 7"],
        "board_channels": [0],
        "pre_config_times": [datetime.datetime(2024, 6, 28, 3, 3, 59)],
        "intermediate_config_times": [
            [
                datetime.datetime(2024, 6, 28, 3, 7, 45),
                datetime.datetime(2024, 6, 28, 3, 12, 35),
                datetime.datetime(2024, 6, 28, 3, 18, 2),
            ],
                                      ],
        "post_config_times": [datetime.datetime(2024, 6, 28, 3, 22, 20)],
        "config_before": True,
        "fluence": 1.5E+7,
        "fluence_type": "Ar",
        "pixel_TMR": "on",
    },
    {
        "name": "Run_6",
        "start": datetime.datetime(2024, 6, 28, 3, 24, 25),
        "stop": datetime.datetime(2024, 6, 28, 3, 42, 34),
        "boards": ["ET2.01 Bare Board 7"],
        "board_channels": [0],
        "pre_config_times": [datetime.datetime(2024, 6, 28, 3, 24, 21)],
        "intermediate_config_times": [
            [
                datetime.datetime(2024, 6, 28, 3, 28, 4),
                datetime.datetime(2024, 6, 28, 3, 33, 32),
                datetime.datetime(2024, 6, 28, 3, 37, 9),
            ],
                                      ],
        "post_config_times": [datetime.datetime(2024, 6, 28, 3, 42, 43)],
        "config_before": False,
        "fluence": 1.5E+7,
        "fluence_type": "Ar",
        "pixel_TMR": "on",
    },
    {
        "name": "Run_7",
        "start": datetime.datetime(2024, 6, 28, 3, 45, 0),
        "stop": datetime.datetime(2024, 6, 28, 4, 3, 10),
        "boards": ["ET2.01 Bare Board 7"],
        "board_channels": [0],
        "pre_config_times": [datetime.datetime(2024, 6, 28, 3, 44, 57)],
        "intermediate_config_times": [
            [
                datetime.datetime(2024, 6, 28, 3, 50, 16),
                datetime.datetime(2024, 6, 28, 3, 54, 32),
                datetime.datetime(2024, 6, 28, 3, 58, 44),
            ],
                                      ],
        "post_config_times": [datetime.datetime(2024, 6, 28, 4, 3, 19)],
        "config_before": False,  # What does it mean "calibrated 4 running pixels"?
        "fluence": 1.5E+7,
        "fluence_type": "Ar",
        "pixel_TMR": "on",
    },
    {
        "name": "Run_8",
        "start": datetime.datetime(2024, 6, 28, 4, 4, 39),
        "stop": datetime.datetime(2024, 6, 28, 4, 22, 49),
        "boards": ["ET2.01 Bare Board 7"],
        "board_channels": [0],
        "pre_config_times": [datetime.datetime(2024, 6, 28, 4, 4, 34)],
        "intermediate_config_times": [
            [
                datetime.datetime(2024, 6, 28, 4, 9, 30),
                datetime.datetime(2024, 6, 28, 4, 16, 44),
                datetime.datetime(2024, 6, 28, 4, 19, 48),
            ],
                                      ],
        "post_config_times": [datetime.datetime(2024, 6, 28, 4, 22, 57)],
        "config_before": False,
        "fluence": 1.5E+7,
        "fluence_type": "Ar",
        "pixel_TMR": "on",
    },
    {
        "name": "Run_9",
        "start": datetime.datetime(2024, 6, 28, 4, 24, 37),
        "stop": datetime.datetime(2024, 6, 28, 4, 42, 44),
        "boards": ["ET2.01 Bare Board 7"],
        "board_channels": [0],
        "pre_config_times": [datetime.datetime(2024, 6, 28, 4, 24, 31)],
        "intermediate_config_times": [
            [
                datetime.datetime(2024, 6, 28, 4, 28, 21),
                datetime.datetime(2024, 6, 28, 4, 33, 8),
                datetime.datetime(2024, 6, 28, 4, 35, 21),
                datetime.datetime(2024, 6, 28, 4, 38, 30),
            ],
                                      ],
        "post_config_times": [datetime.datetime(2024, 6, 28, 4, 43, 0)],
        "config_before": False,
        "fluence": 1.5E+7,
        "fluence_type": "Ar",
        "pixel_TMR": "on",
    },
    {
        "name": "Run_10",
        "start": datetime.datetime(2024, 6, 28, 4, 46, 44),
        "stop": datetime.datetime(2024, 6, 28, 5, 4, 53),
        "boards": ["ET2.01 Bare Board 7"],
        "board_channels": [0],
        "pre_config_times": [datetime.datetime(2024, 6, 28, 4, 46, 36)],
        "intermediate_config_times": [
            [
                datetime.datetime(2024, 6, 28, 4, 51, 21),
                datetime.datetime(2024, 6, 28, 4, 57, 58),
                datetime.datetime(2024, 6, 28, 5, 1, 31),
            ],
                                      ],
        "post_config_times": [datetime.datetime(2024, 6, 28, 5, 5, 2)],
        "config_before": True,
        "fluence": 1.5E+7,
        "fluence_type": "Ar",
        "pixel_TMR": "on",
    },
    {
        "name": "Run_11",
        "start": datetime.datetime(2024, 6, 28, 5, 6, 58),
        "stop": datetime.datetime(2024, 6, 28, 5, 25, 7),
        "boards": ["ET2.01 Bare Board 7"],
        "board_channels": [0],
        "pre_config_times": [datetime.datetime(2024, 6, 28, 5, 6, 56)],
        "intermediate_config_times": [
            [
                datetime.datetime(2024, 6, 28, 5, 8, 56),
                datetime.datetime(2024, 6, 28, 5, 11, 51),
                datetime.datetime(2024, 6, 28, 5, 14, 28),
                datetime.datetime(2024, 6, 28, 5, 16, 10),
                datetime.datetime(2024, 6, 28, 5, 18, 56),
                datetime.datetime(2024, 6, 28, 5, 21, 26),
                datetime.datetime(2024, 6, 28, 5, 23, 33),
            ],
                                      ],
        "post_config_times": [datetime.datetime(2024, 6, 28, 5, 26, 7)],
        "config_before": False,
        "fluence": 1.5E+7,
        "fluence_type": "Ar",
        "pixel_TMR": "on",
    },
    {
        "name": "Run_12",
        "start": datetime.datetime(2024, 6, 28, 5, 27, 18),
        "stop": datetime.datetime(2024, 6, 28, 5, 45, 28),
        "boards": ["ET2.01 Bare Board 7"],
        "board_channels": [0],
        "pre_config_times": [datetime.datetime(2024, 6, 28, 5, 27, 14)],
        "intermediate_config_times": [None],
        "post_config_times": [datetime.datetime(2024, 6, 28, 5, 46, 38)],
        "config_before": False,
        "fluence": 1.5E+7,
        "fluence_type": "Ar",
        "pixel_TMR": "on",
    },
    {
        "name": "Run_13",
        "start": datetime.datetime(2024, 6, 28, 6, 1, 5),
        "stop": datetime.datetime(2024, 6, 28, 6, 8, 6),
        "boards": ["ET2.01 Bare Board 7"],
        "board_channels": [0],
        "pre_config_times": [datetime.datetime(2024, 6, 28, 6, 0, 11)],
        "intermediate_config_times": [
            [
                datetime.datetime(2024, 6, 28, 6, 2, 28),
                datetime.datetime(2024, 6, 28, 6, 4, 41),
            ],
                                      ],
        "post_config_times": [datetime.datetime(2024, 6, 28, 6, 8, 15)],
        "config_before": True,
        "fluence": 5E+6,
        "fluence_type": "Xe",
        "pixel_TMR": "on",
    },
    {
        "name": "Run_14",
        "start": datetime.datetime(2024, 6, 28, 6, 9, 13),
        "stop": datetime.datetime(2024, 6, 28, 6, 15, 35),
        "boards": ["ET2.01 Bare Board 7"],
        "board_channels": [0],
        "pre_config_times": [datetime.datetime(2024, 6, 28, 6, 8, 51)],
        "intermediate_config_times": [
            [
                datetime.datetime(2024, 6, 28, 6, 10, 3),
                datetime.datetime(2024, 6, 28, 6, 12, 8),
                datetime.datetime(2024, 6, 28, 6, 13, 41),
            ],
                                      ],
        "post_config_times": [datetime.datetime(2024, 6, 28, 6, 15, 51)],
        "config_before": False,
        "fluence": 5E+6,
        "fluence_type": "Xe",
        "pixel_TMR": "on",
    },
    {
        "name": "Run_15",
        "start": datetime.datetime(2024, 6, 28, 6, 18, 25),
        "stop": datetime.datetime(2024, 6, 28, 6, 25, 3),
        "boards": ["ET2.01 Bare Board 7"],
        "board_channels": [0],
        "pre_config_times": [datetime.datetime(2024, 6, 28, 6, 17, 20)],
        "intermediate_config_times": [
            [
                datetime.datetime(2024, 6, 28, 6, 20, 13),
                datetime.datetime(2024, 6, 28, 6, 21, 20),
                datetime.datetime(2024, 6, 28, 6, 22, 27),
            ],
                                      ],
        "post_config_times": [datetime.datetime(2024, 6, 28, 6, 26, 16)],
        "config_before": False,  # What does it mean "calibrated 4 running pixels"?
        "fluence": 5E+6,
        "fluence_type": "Xe",
        "pixel_TMR": "on",
    },
    {
        "name": "Run_16",
        "start": datetime.datetime(2024, 6, 28, 6, 27, 28),
        "stop": datetime.datetime(2024, 6, 28, 6, 33, 50),  # Wrong in log
        "boards": ["ET2.01 Bare Board 7"],
        "board_channels": [0],
        "pre_config_times": [datetime.datetime(2024, 6, 28, 6, 27, 15)],
        "intermediate_config_times": [
            [
                datetime.datetime(2024, 6, 28, 6, 29, 7),
                datetime.datetime(2024, 6, 28, 6, 31, 9),
                datetime.datetime(2024, 6, 28, 6, 32, 42),
            ],
                                      ],
        "post_config_times": [datetime.datetime(2024, 6, 28, 6, 34, 6)],
        "config_before": False,
        "fluence": 5E+6,
        "fluence_type": "Xe",
        "pixel_TMR": "on",
    },
    {
        "name": "Run_17",
        "start": datetime.datetime(2024, 6, 28, 6, 35, 52),
        "stop": datetime.datetime(2024, 6, 28, 6, 42, 14),
        "boards": ["ET2.01 Bare Board 7"],
        "board_channels": [0],
        "pre_config_times": [datetime.datetime(2024, 6, 28, 6, 35, 45)],
        "intermediate_config_times": [
            [
                datetime.datetime(2024, 6, 28, 6, 37, 45),
                datetime.datetime(2024, 6, 28, 6, 40, 17),
            ],
                                      ],
        "post_config_times": [datetime.datetime(2024, 6, 28, 6, 42, 35)],
        "config_before": False,
        "fluence": 5E+6,
        "fluence_type": "Xe",
        "pixel_TMR": "on",
    },
    {
        "name": "Run_18",
        "start": datetime.datetime(2024, 6, 28, 6, 43, 56),
        "stop": datetime.datetime(2024, 6, 28, 6, 50, 18),
        "boards": ["ET2.01 Bare Board 7"],
        "board_channels": [0],
        "pre_config_times": [datetime.datetime(2024, 6, 28, 6, 43, 50)],
        "intermediate_config_times": [
            [
                datetime.datetime(2024, 6, 28, 6, 45, 50),
                datetime.datetime(2024, 6, 28, 6, 47, 18),
            ],
                                      ],
        "post_config_times": [datetime.datetime(2024, 6, 28, 6, 50, 34)],
        "config_before": False,
        "fluence": 5E+6,
        "fluence_type": "Xe",
        "pixel_TMR": "on",
    },
    {
        "name": "Run_19",
        "start": datetime.datetime(2024, 6, 28, 6, 51, 28),  # Wrong in log
        "stop": datetime.datetime(2024, 6, 28, 6, 57, 50),
        "boards": ["ET2.01 Bare Board 7"],
        "board_channels": [0],
        "pre_config_times": [datetime.datetime(2024, 6, 28, 6, 51, 20)],
        "intermediate_config_times": [
            [
                datetime.datetime(2024, 6, 28, 6, 53, 9),
                datetime.datetime(2024, 6, 28, 6, 54, 51),
            ],
                                      ],
        "post_config_times": [datetime.datetime(2024, 6, 28, 6, 58, 21)],
        "config_before": False,
        "fluence": 5E+6,
        "fluence_type": "Xe",
        "pixel_TMR": "on",
    },
    {
        "name": "Run_20",
        "start": datetime.datetime(2024, 6, 28, 7, 13, 47),  # Wrong in log
        "stop": datetime.datetime(2024, 6, 28, 7, 20, 9),
        "boards": ["ET2.01 Bare Board 7"],
        "board_channels": [0],
        "pre_config_times": [datetime.datetime(2024, 6, 28, 7, 13, 33)],
        "intermediate_config_times": [
            [
                datetime.datetime(2024, 6, 28, 7, 15, 20),
                datetime.datetime(2024, 6, 28, 7, 17, 29),
            ],
                                      ],
        "post_config_times": [datetime.datetime(2024, 6, 28, 7, 20, 24)],
        "config_before": True,
        "fluence": 5E+6,
        "fluence_type": "Xe",
        "pixel_TMR": "off",
    },
    {
        "name": "Run_21",
        "start": datetime.datetime(2024, 6, 28, 7, 21, 41),
        "stop": datetime.datetime(2024, 6, 28, 7, 28, 5),
        "boards": ["ET2.01 Bare Board 7"],
        "board_channels": [0],
        "pre_config_times": [datetime.datetime(2024, 6, 28, 7, 20, 58)],
        "intermediate_config_times": [
            [
                datetime.datetime(2024, 6, 28, 7, 24, 7),
                datetime.datetime(2024, 6, 28, 7, 25, 28),
                datetime.datetime(2024, 6, 28, 7, 26, 43),
            ],
                                      ],
        "post_config_times": [datetime.datetime(2024, 6, 28, 7, 28, 21)],
        "config_before": False,
        "fluence": 5E+6,
        "fluence_type": "Xe",
        "pixel_TMR": "off", # Is TMR on or off?
    },
    {
        "name": "Run_22",
        "start": datetime.datetime(2024, 6, 28, 7, 28, 56),
        "stop": datetime.datetime(2024, 6, 28, 7, 35, 19),
        "boards": ["ET2.01 Bare Board 7"],
        "board_channels": [0],
        "pre_config_times": [datetime.datetime(2024, 6, 28, 7, 28, 53)],
        "intermediate_config_times": [
            [
                datetime.datetime(2024, 6, 28, 7, 31, 48),
                datetime.datetime(2024, 6, 28, 7, 33, 12),
            ],
                                      ],
        "post_config_times": [datetime.datetime(2024, 6, 28, 7, 35, 41)],
        "config_before": False,
        "fluence": 5E+6,
        "fluence_type": "Xe",
        "pixel_TMR": "off", # Is TMR on or off?
    },
    {
        "name": "Run_23",
        "start": datetime.datetime(2024, 6, 28, 7, 36, 9),
        "stop": datetime.datetime(2024, 6, 28, 7, 42, 33),
        "boards": ["ET2.01 Bare Board 7"],
        "board_channels": [0],
        "pre_config_times": [datetime.datetime(2024, 6, 28, 7, 36, 7)],
        "intermediate_config_times": [
            [
                datetime.datetime(2024, 6, 28, 7, 38, 56),
            ],
                                      ],
        "post_config_times": [datetime.datetime(2024, 6, 28, 7, 42, 54)],
        "config_before": False,
        "fluence": 5E+6,
        "fluence_type": "Xe",
        "pixel_TMR": "off", # Is TMR on or off?
    },
    {
        "name": "Run_24",
        "start": datetime.datetime(2024, 6, 28, 7, 43, 49),
        "stop": datetime.datetime(2024, 6, 28, 7, 49, 58),
        "boards": ["ET2.01 Bare Board 7"],
        "board_channels": [0],
        "pre_config_times": [datetime.datetime(2024, 6, 28, 7, 43, 32)],
        "intermediate_config_times": [
            [
                datetime.datetime(2024, 6, 28, 7, 46, 59),
            ],
                                      ],
        "post_config_times": [datetime.datetime(2024, 6, 28, 7, 50, 11)],
        "config_before": False,
        "fluence": 5E+6,
        "fluence_type": "Xe",
        "pixel_TMR": "off", # Is TMR on or off?
    },
    {
        "name": "Run_25",
        "start": datetime.datetime(2024, 6, 28, 7, 51, 0),
        "stop": datetime.datetime(2024, 6, 28, 7, 57, 8),
        "boards": ["ET2.01 Bare Board 7"],
        "board_channels": [0],
        "pre_config_times": [datetime.datetime(2024, 6, 28, 7, 50, 59)],
        "intermediate_config_times": [
            [
                datetime.datetime(2024, 6, 28, 7, 54, 32),
            ],
                                      ],
        "post_config_times": [datetime.datetime(2024, 6, 28, 7, 57, 17)],
        "config_before": False,
        "fluence": 5E+6,
        "fluence_type": "Xe",
        "pixel_TMR": "off", # Is TMR on or off?
    },
]
extra_run_info = [
]
test_run_info = [
]
run_info_backup = run_info
joined_config_analysis = {
}

# Power Monitoring

In [ ]:
power_directory = output_dir/"PowerPlots"
power_directory.mkdir(exist_ok=True)

with sqlite3.connect(power_file) as sqlite3_connection:
    data_df = pandas.read_sql('SELECT * FROM power_v2', sqlite3_connection, index_col=None)
    data_df['Time'] = pandas.to_datetime(data_df['timestamp'], format='mixed')

    # Remove data that is outside the range of the campaign
    data_df = data_df.loc[data_df['Time'] >= start_time]
    data_df = data_df.loc[data_df['Time'] <= end_time].copy()

    data_df['V'] = data_df['V'].astype(float)
    data_df['I'] = data_df['I'].astype(float)

    print("Timestamps in power database file cover range:")
    print("Min:", data_df['Time'].min())
    print("Max", data_df['Time'].max())

    #print(data_df)

    vref_df = data_df.loc[data_df['Channel'] == 'VRef']
    #ws_df = data_df.loc[data_df['Channel'] == 'WS']
    wsana_df = data_df.loc[data_df['Channel'] == 'WSAnalog']
    wsdig_df = data_df.loc[data_df['Channel'] == 'WSDigital']

## VRef

Plot VRef over time so we can find the range of times where we can be confident the data is ok

In [ ]:
plotVRefPower(vref_df, 'SEU - VRef over Time', power_directory, True)

## ET2 Bare 7

In [ ]:
board = "ET2.01 Bare Board 7"
plotBoardPower(board, power_connections[board], data_df, f'SEU - {board} Power over Time', power_directory, True)

## Waveform Sampler

In [ ]:
plotWSPower(wsana_df, 'SEU - Waveform Sampler Analog Power over Time', power_directory, True, filename = "WaveformSamplerAnalog")

In [ ]:
plotWSPower(wsdig_df, 'SEU - Waveform Sampler Digital Power over Time', power_directory, True, filename = "WaveformSamplerDigital")

## Per Run Plotting

In [ ]:
run_power_dir = power_directory/"PerRun"
run_power_dir.mkdir(exist_ok=True)

for run_idx_to_plot in range(len(run_info)):
    if run_idx_to_plot == 0:
        makePerRunPlots(
            data_df,
            run_info[run_idx_to_plot],
            run_power_dir,
            power_connections,
            extra_run_info = extra_run_info,
            test_run_info = test_run_info,
        )
    else:
        makePerRunPlots(
            data_df,
            run_info[run_idx_to_plot],
            run_power_dir,
            power_connections,
            previous_run_info = run_info[run_idx_to_plot - 1],
            extra_run_info = extra_run_info,
            test_run_info = test_run_info,
        )

for this_run_info in extra_run_info:
    makePerRunPlots(
        data_df,
        this_run_info,
        run_power_dir,
        power_connections,
        all_run_info = run_info,
        extra_run_info = extra_run_info,
        test_run_info = test_run_info,
    )


# Baseline History

In [ ]:
baseline_history_file = base_directory/"ETROC-History"/"BaselineHistory.sqlite"
baseline_directory = output_dir/"Baselines"
baseline_directory.mkdir(exist_ok=True)

col_list = [8, 2, 8, 2]
row_list = [0, 0, 2, 2]
scan_list = list(zip(row_list, col_list))
chip_df = {}

times_to_plot = {
}
for run in run_info:
    times_to_plot[run["name"] + " Start"] = run["start"]
    #times_to_plot[run["name"] + " Stop"] = run["stop"]

with sqlite3.connect(baseline_history_file) as sqlite3_connection:
    baseline_df = pandas.read_sql('SELECT * FROM baselines', sqlite3_connection, index_col=None)
    baseline_df['Time'] = pandas.to_datetime(baseline_df['timestamp'], format='mixed')

    # Remove data that is outside the range of the campaign
    baseline_df = baseline_df.loc[baseline_df['Time'] >= (start_time - datetime.timedelta(minutes=12))]
    baseline_df = baseline_df.loc[baseline_df['Time'] <= end_time].copy()

    for chip_name in chip_names:
        chip_df[chip_name] = baseline_df.loc[baseline_df["chip_name"] == chip_name].copy()

        plotPixelsOverTime(chip_df[chip_name], 'baseline', f"Baselines of {chip_name.replace("_", " ")}", scan_list, baseline_directory/f"{chip_name}_Baseline.pdf", times_to_plot = {}, show = False)
        plotPixelsOverTime(chip_df[chip_name], 'noise_width', f"Noise Widths of {chip_name.replace("_", " ")}", scan_list, baseline_directory/f"{chip_name}_NoiseWidth.pdf", times_to_plot = {}, show = False)

        plotPixelsOverTime(chip_df[chip_name], 'baseline', f"Baselines of {chip_name.replace("_", " ")}", scan_list, baseline_directory/f"{chip_name}_Baseline_runInfo.pdf", times_to_plot = times_to_plot, show = True)
        plotPixelsOverTime(chip_df[chip_name], 'noise_width', f"Noise Widths of {chip_name.replace("_", " ")}", scan_list, baseline_directory/f"{chip_name}_NoiseWidth_runInfo.pdf", times_to_plot = times_to_plot, show = True)


# ADC

In [ ]:
adc_history_file = base_directory/"ETROC-History"/"ADCHistory.sqlite"
adc_directory = output_dir/"ADC"
adc_directory.mkdir(exist_ok=True)

times_to_plot = {
}
for run in run_info:
    times_to_plot[run["name"] + " Start"] = run["start"]
    #times_to_plot[run["name"] + " Stop"] = run["stop"]

active_channels = [1]

with sqlite3.connect(adc_history_file) as sqlite3_connection:
    adc_df = pandas.read_sql('SELECT * FROM adc', sqlite3_connection, index_col=None)
    adc_df['Time'] = pandas.to_datetime(adc_df['timestamp'], format='mixed')

    # Remove data that is outside the range of the campaign
    adc_df = adc_df.loc[adc_df['Time'] >= (start_time - datetime.timedelta(minutes=12))]
    adc_df = adc_df.loc[adc_df['Time'] <= end_time].copy()

    temp_df = adc_df.loc[adc_df['channel'] == 8].copy()
    plotDFOverTime(temp_df, 'calibrated', "AD5593R Temperature over Time", "Temp [C]", adc_directory/"AD5593R_Temperature.pdf", times_to_plot = {}, show = False)
    plotDFOverTime(temp_df, 'calibrated', "AD5593R Temperature over Time", "Temp [C]", adc_directory/"AD5593R_Temperature_runInfo.pdf", times_to_plot = times_to_plot, show = True)

    plotADCOverTime(adc_df, active_channels, f"VTemp of {chip_names[0].replace("_", " ")}", adc_directory/"VTemp.pdf", times_to_plot = {}, show = False)
    plotADCOverTime(adc_df, active_channels, f"VTemp of {chip_names[0].replace("_", " ")}", adc_directory/"VTemp_runInfo.pdf", times_to_plot = times_to_plot, show = True)


# I2C Config

In [ ]:
import pickle
import numpy
import copy
import tempfile
import shutil

from math import ceil

config_compare_dir = output_dir/"I2CConfiguration"
config_compare_dir.mkdir(exist_ok=True)
base_config_compare_dir = config_compare_dir

base_bitmap = {
}
for idx in range(32):
    base_bitmap[f'PeriCfg{idx}']  = [0 for _ in range(8)]
for idx in range(16):
    base_bitmap[f'PeriStat{idx}'] = [0 for _ in range(8)]
for idx in range(32):
    base_bitmap[f'PixCfg{idx}']   = [0 for _ in range(8)]
for idx in range(8):
    base_bitmap[f'PixStat{idx}']  = [0 for _ in range(8)]

## Optional run_info filtering
In order to only process a few runs

In [ ]:
## Filter run info
run_info = []

for run in run_info_backup:
    if run["fluence_type"] != "Kr":
        continue
    if run["pixel_TMR"] != "off":
        continue
    run_info.append(run)

extra_file_name = "Kr_TMRoff"

config_compare_dir = base_config_compare_dir/extra_file_name
config_compare_dir.mkdir(exist_ok=True)

print(run_info)
for info in run_info:
    print(f'  {info["name"]}')

In [ ]:
## Filter run info
run_info = []

for run in run_info_backup:
    if run["fluence_type"] != "Kr":
        continue
    if run["pixel_TMR"] != "on":
        continue
    run_info.append(run)

extra_file_name = "Kr_TMRon"

config_compare_dir = base_config_compare_dir/extra_file_name
config_compare_dir.mkdir(exist_ok=True)

print(run_info)
for info in run_info:
    print(f'  {info["name"]}')

In [ ]:
## Filter run info
run_info = []

for run in run_info_backup:
    if run["fluence_type"] != "Ar":
        continue
    if run["pixel_TMR"] != "on":
        continue
    run_info.append(run)

extra_file_name = "Ar_TMRon"

config_compare_dir = base_config_compare_dir/extra_file_name
config_compare_dir.mkdir(exist_ok=True)

print(run_info)
for info in run_info:
    print(f'  {info["name"]}')

In [ ]:
## Filter run info
run_info = []

for run in run_info_backup:
    if run["fluence_type"] != "Xe":
        continue
    if run["pixel_TMR"] != "on":
        continue
    run_info.append(run)

extra_file_name = "Xe_TMRon"

config_compare_dir = base_config_compare_dir/extra_file_name
config_compare_dir.mkdir(exist_ok=True)

print(run_info)
for info in run_info:
    print(f'  {info["name"]}')

In [ ]:
## Filter run info
run_info = []

for run in run_info_backup:
    if run["fluence_type"] != "Xe":
        continue
    if run["pixel_TMR"] != "off":
        continue
    run_info.append(run)

extra_file_name = "Xe_TMRoff"

config_compare_dir = base_config_compare_dir/extra_file_name
config_compare_dir.mkdir(exist_ok=True)

print(run_info)
for info in run_info:
    print(f'  {info["name"]}')

In [ ]:
run_info = run_info_backup
extra_file_name = ""
config_compare_dir = base_config_compare_dir

## Do config comparison

In [ ]:
accumulated_status_map = {}
accumulated_config_map = {}
accumulated_bitmap = {}

extra_bitmap = {}

counter = 0
for this_run_idx in range(len(run_info)):
    counter += 1
    #if counter == 2:
    #    break
    #if counter < 4:
    #    continue
    this_run_info = run_info[this_run_idx]
    this_run_name = this_run_info["name"]

    this_run_dir = base_config_compare_dir/this_run_name
    this_run_dir.mkdir(exist_ok=True)

    for board_idx in range(len(this_run_info["boards"])):
        board_name = this_run_info["boards"][board_idx]

        if board_name not in accumulated_bitmap:
            accumulated_bitmap[board_name] = copy.deepcopy(base_bitmap)

        pre_config_time  = this_run_info[ "pre_config_times"][board_idx]
        post_config_time = this_run_info["post_config_times"][board_idx]

        if post_config_time is None:
            print(f'No post config time for board {board_name} for run {this_run_name}')
            continue

        if pre_config_time is None:
            if "config_before" in this_run_info and not this_run_info["config_before"]:
                for offset in range(this_run_idx):
                    previous_run_info = run_info[this_run_idx - 1 - offset]

                    previous_board_idx = None
                    for idx in range(len(previous_run_info["boards"])):
                        if board_name == previous_run_info["boards"][idx]:
                            previous_board_idx = idx
                            break

                    if previous_board_idx is None:
                        continue  #  TODO: Perhaps this should be a break

                    pre_config_time = previous_run_info["post_config_times"][previous_board_idx]

                    if pre_config_time is not None:
                        break

                    if "config_before" in previous_run_info and not previous_run_info["config_before"]:
                        continue
                    break

            if pre_config_time is None:
                print(f'No pre config time for board {board_name} for run {this_run_name}')
                continue

        pre_time_tag  =  pre_config_time.isoformat().replace(":", "-")
        post_time_tag = post_config_time.isoformat().replace(":", "-")

        pre_time_file  = list(config_directory.glob(f'{pre_time_tag}*'))
        post_time_file = list(config_directory.glob(f'{post_time_tag}*'))
        if len(pre_time_file) == 0:
            print(f"Couldn't find the pre-time configuration file for board {board_name} for run {this_run_name}")
            continue
        if len(post_time_file) == 0:
            print(f"Couldn't find the post-time configuration file for board {board_name} for run {this_run_name}")
            continue
        if len(pre_time_file) > 1:
            print(f"Found too many pre-time configuration files for board {board_name} for run {this_run_name}")
            continue
        if len(post_time_file) > 1:
            print(f"Found too many post-time configuration files for board {board_name} for run {this_run_name}")
            continue
        pre_time_file  =  pre_time_file[0]
        post_time_file = post_time_file[0]

        print(f'{this_run_name} - {board_name}')

        changed_registers = diff_chip_configs(pre_time_file, post_time_file)
        save_changed_config(changed_registers, this_run_dir, f"changedRegisters_{board_name.replace(' ', '_')}")

        status_map, config_map = get_pixel_bitflip_map(changed_registers)
        plot_map(status_map, f"Pixel Status Bit Flips on {board_name} during Run {this_run_name}", 'Bit Flips', this_run_dir/f"{board_name}_Status.pdf")
        plot_map(config_map, f"Pixel Config Bit Flips on {board_name} during Run {this_run_name}", 'Bit Flips', this_run_dir/f"{board_name}_Config.pdf")

        this_bitmap = copy.deepcopy(base_bitmap)
        other_bitmap = {}
        fill_bitmap(changed_registers, this_bitmap, other_bitmap)
        save_bitmap_table(this_bitmap, this_run_dir, f"{board_name}")
        save_bitmap_bitflips(changed_registers, this_run_dir, f"{board_name}")

        for key in other_bitmap:
            if key not in extra_bitmap:
                extra_bitmap[key] = [0 for _ in range(8)]
            for idx in range(8):
                extra_bitmap[key][idx] += other_bitmap[key][idx]
        for key in this_bitmap:
            for idx in range(8):
                accumulated_bitmap[board_name][key][idx] += this_bitmap[key][idx]

        if board_name not in accumulated_status_map:
            accumulated_status_map[board_name] = status_map
        else:
            for row in range(16):
                for col in range(16):
                    accumulated_status_map[board_name][row][col] += status_map[row][col]
        if board_name not in accumulated_config_map:
            accumulated_config_map[board_name] = config_map
        else:
            for row in range(16):
                for col in range(16):
                    accumulated_config_map[board_name][row][col] += config_map[row][col]

#print(extra_bitmap)
#print(accumulated_bitmap)

if len(extra_bitmap) != 0:
    with open(config_compare_dir/("extra_registers_bitflips.pickle"), "wb") as file:
        pickle.dump(extra_bitmap, file)

    with open(config_compare_dir/("extra_registers_bitflips.txt"), "w") as file:
        file.write("The bits in unnamed registers saw the following amount of bit flips:")
        for register in extra_bitmap:
            file.write(f" - {register}: {extra_bitmap[register]}")

total_bitmap = copy.deepcopy(base_bitmap)
for board_name in accumulated_bitmap:
    save_bitmap_table(accumulated_bitmap[board_name], config_compare_dir, f"{board_name}")
    for key in accumulated_bitmap[board_name]:
        for idx in range(8):
            total_bitmap[key][idx] += accumulated_bitmap[board_name][key][idx]
save_bitmap_table(total_bitmap, config_compare_dir, "Total")

for board_name in accumulated_status_map:
    plot_map(accumulated_status_map[board_name], f"Pixel Status Bit Flips on {board_name} over all Runs", 'Bit Flips', config_compare_dir/f"{board_name}_Status.pdf")
for board_name in accumulated_config_map:
    plot_map(accumulated_config_map[board_name], f"Pixel Config Bit Flips on {board_name} over all Runs", 'Bit Flips', config_compare_dir/f"{board_name}_Config.pdf")


## Do the Joined Config analysis

In [ ]:
for ion in joined_config_analysis:
    dir_name = f"{ion}_joined"

    this_run_dir = base_config_compare_dir/dir_name
    this_run_dir.mkdir(exist_ok=True)

    for board_idx in range(len(run_info[0]["boards"])):
        board_name = run_info[0]["boards"][board_idx]

        pre_config_time  = joined_config_analysis[ion][0][board_idx]
        post_config_time = joined_config_analysis[ion][1][board_idx]

        pre_time_tag  =  pre_config_time.isoformat().replace(":", "-")
        post_time_tag = post_config_time.isoformat().replace(":", "-")

        pre_time_file  = list(config_directory.glob(f'{pre_time_tag}*'))
        post_time_file = list(config_directory.glob(f'{post_time_tag}*'))
        if len(pre_time_file) == 0:
            print(f"Couldn't find the pre-time configuration file for board {board_name} for run {this_run_name}")
            continue
        if len(post_time_file) == 0:
            print(f"Couldn't find the post-time configuration file for board {board_name} for run {this_run_name}")
            continue
        if len(pre_time_file) > 1:
            print(f"Found too many pre-time configuration files for board {board_name} for run {this_run_name}")
            continue
        if len(post_time_file) > 1:
            print(f"Found too many post-time configuration files for board {board_name} for run {this_run_name}")
            continue
        pre_time_file  =  pre_time_file[0]
        post_time_file = post_time_file[0]

        changed_registers = diff_chip_configs(pre_time_file, post_time_file)
        save_changed_config(changed_registers, this_run_dir, f"changedRegisters_{board_name.replace(' ', '_')}")

        status_map, config_map = get_pixel_bitflip_map(changed_registers)
        plot_map(status_map, f"Pixel Status Bit Flips on {board_name} during {ion} Runs", 'Bit Flips', this_run_dir/f"{board_name}_Status.pdf")
        plot_map(config_map, f"Pixel Config Bit Flips on {board_name} during {ion} Runs", 'Bit Flips', this_run_dir/f"{board_name}_Config.pdf")

        this_bitmap = copy.deepcopy(base_bitmap)
        other_bitmap = {}
        fill_bitmap(changed_registers, this_bitmap, other_bitmap)
        save_bitmap_table(this_bitmap, this_run_dir, f"{board_name}")


# Data

In [ ]:
from beamtest_analysis_helper import DecodeBinary
from beamtest_analysis_helper import return_hist, return_event_hist, return_crc_hist, plot_1d_TDC_histograms, plot_1d_event_CRC_histogram, plot_1d_CRC_histogram
from natsort import natsorted

translated_dir = output_dir/"Translated"
translated_dir.mkdir(exist_ok=True)
plot_dir = output_dir/"TDCPlots"
plot_dir.mkdir(exist_ok=True)

## Translate RAW data

In [ ]:
def translateRun(this_run_info: dict, save_dir: Path):
    this_run_name = this_run_info["name"]
    this_run_dir: Path = base_directory/this_run_name

    if not this_run_dir.is_dir():
        raise RuntimeError(f"The directory for run {this_run_name} does not exist. Expected: {this_run_dir}")

    print(f"Translating data for run {this_run_name}")
    this_run_files = natsorted(list(this_run_dir.glob("loop*/*")))

    decoder = DecodeBinary(
        firmware_key=0b0001,
        board_id = [0x17f0f, 0x17f0f, 0x17f0f, 0x17f0f],
        file_list = this_run_files,
        save_nem = None,
        skip_filler = True,
    )
    dataframe, event_dataframe, crc_dataframe, filler_dataframe = decoder.decode_files()

    print(f"Saving data for run {this_run_name}")

    #dataframe.to_feather(save_dir/f"{this_run_name}.feather")
    with sqlite3.connect(save_dir/f"{this_run_name}.sqlite") as sqlite3_connection:
        dataframe = dataframe.drop(columns=['bcid', 'l1a_counter'])
        dataframe.to_sql('hit_data', sqlite3_connection, if_exists='replace', index=False)
        event_dataframe.to_sql('event_data', sqlite3_connection, if_exists='replace', index=False)
        crc_dataframe.to_sql('crc_data', sqlite3_connection, if_exists='replace', index=False)
        filler_dataframe.to_sql('filler_data', sqlite3_connection, if_exists='replace', index=False)

counter = 0
for this_run_info in run_info:
    counter += 1
    #print(f"Run {counter}")
    #if counter < 12:
    #    continue
    #if counter == 13:
    #    break
    translateRun(this_run_info, translated_dir)

for this_run_info in extra_run_info:
    translateRun(this_run_info, translated_dir)

for this_run_info in test_run_info:
    translateRun(this_run_info, translated_dir)

## Make per run TDC Plots

In [ ]:
def plotRunSummary(this_run_info: dict, translated_dir: Path, save_dir: Path, ps_see_time: datetime.datetime | None = None, print_info: bool = False):
    this_run_name = this_run_info["name"]
    this_run_dir: Path = base_directory/this_run_name
    this_run_plot_dir = save_dir/this_run_name
    this_run_plot_dir.mkdir(exist_ok=True)

    if not this_run_dir.is_dir():
        raise RuntimeError(f"The directory for run {this_run_name} does not exist. Expected: {this_run_dir}")

    #dataframe = pandas.read_feather(translated_dir/f"{this_run_name}.feather")
    with sqlite3.connect(translated_dir/f"{this_run_name}.sqlite") as sqlite3_connection:
        dataframe = pandas.read_sql('SELECT * FROM hit_data', sqlite3_connection, index_col=None)
        event_dataframe = pandas.read_sql('SELECT * FROM event_data', sqlite3_connection, index_col=None)
        crc_dataframe = pandas.read_sql('SELECT * FROM crc_data', sqlite3_connection, index_col=None)

    if len(dataframe) == 0:
        return

    filtered_df = dataframe.loc[(dataframe["tot"] - dataframe["tot"].mean()).abs() > 2*dataframe["tot"].std()].copy()
    off_peak_hits = len(filtered_df)

    if print_info:
        print(f"All hits on all boards: {len(dataframe)}")
        print(f"Hits not in main TOT peak: {off_peak_hits}")

    board_labels = this_run_info["boards"]
    board_names = [i.replace(" ", "_") for i in board_labels]
    board_channels = this_run_info["board_channels"]
    this_run_hists = return_hist(dataframe, board_names, board_channels, hist_bins=[100, 128, 128])
    this_event_hist = return_event_hist(event_dataframe)
    this_crc_hists = return_crc_hist(crc_dataframe, board_names, board_channels)
    filtered_run_hists = return_hist(filtered_df, board_names, board_channels, hist_bins=[100, 128, 128])

    for board_idx in range(len(board_labels)):
        plot_1d_TDC_histograms(this_run_hists, board_names[board_idx], board_names[board_idx], board_labels[board_idx], this_run_plot_dir, save=True, tag="inclusive", fig_tag=", inclusive", slide_friendly=False, event_hist=this_event_hist)
        plot_1d_TDC_histograms(this_run_hists, board_names[board_idx], board_names[board_idx], board_labels[board_idx], this_run_plot_dir, save=True, tag="inclusive_logy", fig_tag=", inclusive", slide_friendly=False, do_logy = True, event_hist=this_event_hist)
        plot_1d_TDC_histograms(this_run_hists, board_names[board_idx], board_names[board_idx], board_labels[board_idx], this_run_plot_dir, save=True, tag="inclusive", fig_tag=", inclusive", slide_friendly=True, event_hist=this_event_hist)
        plot_1d_TDC_histograms(this_run_hists, board_names[board_idx], board_names[board_idx], board_labels[board_idx], this_run_plot_dir, save=True, tag="inclusive_logy", fig_tag=", inclusive", slide_friendly=True, do_logy = True, event_hist=this_event_hist)

        plot_1d_event_CRC_histogram(this_event_hist, this_run_plot_dir, save=True, tag="inclusive", fig_tag=", inclusive", do_logy = False)
        plot_1d_event_CRC_histogram(this_event_hist, this_run_plot_dir, save=True, tag="inclusive_logy", fig_tag=", inclusive", do_logy = True)

        plot_1d_CRC_histogram(this_crc_hists, board_names[board_idx], board_names[board_idx], board_labels[board_idx], this_run_plot_dir, save=True, tag="inclusive", fig_tag=", inclusive", do_logy = False)
        plot_1d_CRC_histogram(this_crc_hists, board_names[board_idx], board_names[board_idx], board_labels[board_idx], this_run_plot_dir, save=True, tag="inclusive_logy", fig_tag=", inclusive", do_logy = True)

        if off_peak_hits > 0:
            plot_1d_TDC_histograms(filtered_run_hists, board_names[board_idx], board_names[board_idx], board_labels[board_idx], this_run_plot_dir, save=True, tag="filtered_tot_peak", fig_tag=", filtered TOT peak", slide_friendly=False)
            plot_1d_TDC_histograms(filtered_run_hists, board_names[board_idx], board_names[board_idx], board_labels[board_idx], this_run_plot_dir, save=True, tag="filtered_tot_peak_logy", fig_tag=", filtered TOT peak", slide_friendly=False, do_logy = True)
            plot_1d_TDC_histograms(filtered_run_hists, board_names[board_idx], board_names[board_idx], board_labels[board_idx], this_run_plot_dir, save=True, tag="filtered_tot_peak", fig_tag=", filtered TOT peak", slide_friendly=True)
            plot_1d_TDC_histograms(filtered_run_hists, board_names[board_idx], board_names[board_idx], board_labels[board_idx], this_run_plot_dir, save=True, tag="filtered_tot_peak_logy", fig_tag=", filtered TOT peak", slide_friendly=True, do_logy = True)

    #counts = event_dataframe['hamming_count'].unique()
    #if len(counts) > 1:
    #    for hamming_count in counts:
    #        filtered_event_df = event_dataframe.loc[event_dataframe['hamming_count'] == hamming_count]

    #        print(filtered_event_df)
    #        filtered_df = dataframe[dataframe['evt'].isin(filtered_event_df['evt'])].copy()
    #        print(filtered_df)

    values = dataframe['ea'].unique()
    if len(values) > 1:
        for hamming_code in values:
            filtered_df    = dataframe.loc[dataframe['ea'] == hamming_code].copy()
            filtered_hists = return_hist(filtered_df, board_names, board_channels, hist_bins=[100, 128, 128])

            for board_idx in range(len(board_labels)):
                plot_1d_TDC_histograms(filtered_hists, board_names[board_idx], board_names[board_idx], board_labels[board_idx], this_run_plot_dir, save=True, tag=f"hamming_code_{hamming_code}", fig_tag=f", Hamming Code {hamming_code:#04b}", slide_friendly=False)
                plot_1d_TDC_histograms(filtered_hists, board_names[board_idx], board_names[board_idx], board_labels[board_idx], this_run_plot_dir, save=True, tag=f"hamming_code_{hamming_code}_logy", fig_tag=f", Hamming Code {hamming_code:#04b}", slide_friendly=False, do_logy = True)
                plot_1d_TDC_histograms(filtered_hists, board_names[board_idx], board_names[board_idx], board_labels[board_idx], this_run_plot_dir, save=True, tag=f"hamming_code_{hamming_code}", fig_tag=f", Hamming Code {hamming_code:#04b}", slide_friendly=True)
                plot_1d_TDC_histograms(filtered_hists, board_names[board_idx], board_names[board_idx], board_labels[board_idx], this_run_plot_dir, save=True, tag=f"hamming_code_{hamming_code}_logy", fig_tag=f", Hamming Code {hamming_code:#04b}", slide_friendly=True, do_logy = True)


    run_start = this_run_info['start']
    run_stop = run_start  ## For the very short runs, like the test runs, where we did not define a stop time
    if 'stop' in this_run_info:
        run_stop  = this_run_info['stop']

    ##   It seems that after the PS SEE, data was no longer being recorded, so it doesn't
    ## make sense to filter the events based on the fraction of total events between good
    ## events (before PS SEE) and bad events (after PS SEE)
    #if ps_see_time > run_start and ps_see_time < run_stop:
    #    run_duration = (run_stop - run_start)
    #    percentage_good = (ps_see_time - run_start)/run_duration

    #    total_rows = len(dataframe)
    #    good_rows = floor(total_rows * percentage_good * 0.95)  # Add a 5% safety factor
    #    bad_rows = floor(total_rows * (1 - percentage_good) * 0.95)

    #    good_df = dataframe.iloc[:good_rows]
    #    bad_df = dataframe.iloc[-bad_rows:]

    #    good_run_hists = return_hist(good_df, board_names, board_channels, hist_bins=[100, 128, 128])
    #    bad_run_hists  = return_hist(bad_df, board_names, board_channels, hist_bins=[100, 128, 128])

    #    for board_idx in range(len(board_labels)):
    #        #plot_1d_TDC_histograms(good_run_hists, board_names[board_idx], board_names[board_idx], board_labels[board_idx], this_run_plot_dir, save=True, tag="inclusive_good", fig_tag=" good, inclusive", slide_friendly=False)
    #        plot_1d_TDC_histograms(good_run_hists, board_names[board_idx], board_names[board_idx], board_labels[board_idx], this_run_plot_dir, save=True, tag="inclusive_good", fig_tag=" good, inclusive", slide_friendly=True, event_hist=this_event_hist)
    #        plot_1d_TDC_histograms(good_run_hists, board_names[board_idx], board_names[board_idx], board_labels[board_idx], this_run_plot_dir, save=True, tag="inclusive_logy_good", fig_tag=" good, inclusive", slide_friendly=True, do_logy = True, event_hist=this_event_hist)

    #        #plot_1d_TDC_histograms(bad_run_hists, board_names[board_idx], board_names[board_idx], board_labels[board_idx], this_run_plot_dir, save=True, tag="inclusive_bad", fig_tag=" bad, inclusive", slide_friendly=False)
    #        plot_1d_TDC_histograms(bad_run_hists, board_names[board_idx], board_names[board_idx], board_labels[board_idx], this_run_plot_dir, save=True, tag="inclusive_bad", fig_tag=" bad, inclusive", slide_friendly=True, event_hist=this_event_hist)
    #        plot_1d_TDC_histograms(bad_run_hists, board_names[board_idx], board_names[board_idx], board_labels[board_idx], this_run_plot_dir, save=True, tag="inclusive_logy_bad", fig_tag=" bad, inclusive", slide_friendly=True, do_logy = True, event_hist=this_event_hist)

counter = 0
for this_run_info in run_info:
    counter += 1
    print(f"Run {counter}")
    #if counter < 12:
    #    continue
    #if counter == 13:
    #    break
    plotRunSummary(this_run_info, translated_dir, plot_dir, None, True)

for this_run_info in extra_run_info:
    plotRunSummary(this_run_info, translated_dir, plot_dir, None)

for this_run_info in test_run_info:
    plotRunSummary(this_run_info, translated_dir, plot_dir, None)


# Generate Slides

In [ ]:
slide_dir = output_dir/"SummarySlides"
slide_dir.mkdir(exist_ok=True)

## Overall Summary

In [ ]:
overall_dir = slide_dir/"Overall"
overall_dir.mkdir(exist_ok=True)
makeOverallSummarySlides(
    overall_dir,
    start_time,
    end_time,
    chip_names,
    run_info,
    extra_run_info,
    power_directory,
    baseline_directory,
    scan_list,
    config_compare_dir,
    plot_dir,
    [ps_see_time],
    slide_subtitle = r"Jan 27, 2024",
                         )

## Per Run Summary

In [ ]:
counter = 0
for this_run_info in run_info:
    counter += 1
    print(f"Run {counter}")
    #if counter == 12:
    #    break
    #if counter < 11:
    #    continue

    this_run_dir = slide_dir/this_run_info["name"]
    this_run_dir.mkdir(exist_ok=True)

    # Add call to missing function to create latex slides and convert to pdf

for this_run_info in extra_run_info:
    this_run_dir = slide_dir/this_run_info["name"]
    this_run_dir.mkdir(exist_ok=True)

    # Add call to missing function to create latex slides and convert to pdf